In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

In [2]:
df = pd.read_csv("loan_data.csv")

In [3]:
df.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [4]:
df.shape

(381, 13)

In [5]:
df.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
376,LP002953,Male,Yes,3+,Graduate,No,5703,0.0,128.0,360.0,1.0,Urban,Y
377,LP002974,Male,Yes,0,Graduate,No,3232,1950.0,108.0,360.0,1.0,Rural,Y
378,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
379,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
380,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


In [7]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,381.000000,381.000000,381.000000,370.000000,351.000000
mean,3579.845144,1277.275381,104.986877,340.864865,0.837607
std,1419.813818,2340.818114,28.358464,68.549257,0.369338
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2600.000000,0.000000,90.000000,360.000000,1.000000
50%,3333.000000,983.000000,110.000000,360.000000,1.000000
75%,4288.000000,2016.000000,127.000000,360.000000,1.000000
max,9703.000000,33837.000000,150.000000,480.000000,1.000000


In [8]:
df.isnull().sum()

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [9]:
df = df.drop("Loan_ID",axis=1)

In [10]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [11]:
df = df.dropna(subset=["Gender","Dependents","Loan_Amount_Term"])               

In [12]:
df.shape

(358, 12)

In [13]:
df.isnull().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        20
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [14]:
df["Self_Employed"].unique()

array(['No', 'Yes', nan], dtype=object)

In [15]:
df["Self_Employed"].mode()[0]

'No'

In [16]:
 df["Self_Employed"].fillna(df["Self_Employed"].mode()[0],inplace=True)

C:\Users\Siddhi\AppData\Local\Temp\ipykernel_21268\1960155729.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Self_Employed"].fillna(df["Self_Employed"].mode()[0],inplace=True)


In [17]:
df["Credit_History"].unique()

array([ 1., nan,  0.])

In [18]:
df["Credit_History"].mean()

0.8597560975609756

In [19]:
df["Credit_History"].mode()[0]

1.0

In [20]:
 df["Credit_History"].fillna(df["Credit_History"].mode()[0],inplace=True)

C:\Users\Siddhi\AppData\Local\Temp\ipykernel_21268\3835093474.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Credit_History"].fillna(df["Credit_History"].mode()[0],inplace=True)


In [21]:
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 380
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             358 non-null    object 
 1   Married            358 non-null    object 
 2   Dependents         358 non-null    object 
 3   Education          358 non-null    object 
 4   Self_Employed      358 non-null    object 
 5   ApplicantIncome    358 non-null    int64  
 6   CoapplicantIncome  358 non-null    float64
 7   LoanAmount         358 non-null    float64
 8   Loan_Amount_Term   358 non-null    float64
 9   Credit_History     358 non-null    float64
 10  Property_Area      358 non-null    object 
 11  Loan_Status        358 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 36.4+ KB


In [23]:
df["Gender"].unique()

array(['Male', 'Female'], dtype=object)

In [24]:
df["Dependents"].unique()

array(['1', '0', '2', '3+'], dtype=object)

In [25]:
df["Dependents"].replace('3+' , '4', inplace =True)

C:\Users\Siddhi\AppData\Local\Temp\ipykernel_21268\14962369.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dependents"].replace('3+' , '4', inplace =True)


In [26]:
df["Dependents"].unique()

array(['1', '0', '2', '4'], dtype=object)

In [27]:
df["Married"].unique()

array(['Yes', 'No'], dtype=object)

In [32]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["Gender"]=le.fit_transform(df["Gender"])

In [33]:
df["Gender"]

0      1
1      1
2      1
3      1
4      1
      ..
376    1
377    1
378    0
379    1
380    0
Name: Gender, Length: 358, dtype: int32

In [34]:
df["Married"]=le.fit_transform(df["Married"])

In [35]:
df["Married"]

0      1
1      1
2      1
3      0
4      1
      ..
376    1
377    1
378    0
379    1
380    0
Name: Married, Length: 358, dtype: int32

In [37]:
df["Education"]=le.fit_transform(df["Education"])

In [38]:
df["Self_Employed"]=le.fit_transform(df["Self_Employed"])

In [40]:
df["Property_Area"]=le.fit_transform(df["Property_Area"])

In [41]:
df["Loan_Status"]=le.fit_transform(df["Loan_Status"])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 380
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             358 non-null    int32  
 1   Married            358 non-null    int32  
 2   Dependents         358 non-null    int32  
 3   Education          358 non-null    int32  
 4   Self_Employed      358 non-null    int32  
 5   ApplicantIncome    358 non-null    int64  
 6   CoapplicantIncome  358 non-null    float64
 7   LoanAmount         358 non-null    float64
 8   Loan_Amount_Term   358 non-null    float64
 9   Credit_History     358 non-null    float64
 10  Property_Area      358 non-null    int32  
 11  Loan_Status        358 non-null    int32  
dtypes: float64(4), int32(7), int64(1)
memory usage: 26.6 KB


In [43]:
X = df.drop("Loan_Status",axis = 1)

In [44]:
Y = df["Loan_Status"]

In [45]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
1,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
2,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
3,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
4,1,1,0,1,0,2333,1516.0,95.0,360.0,1.0,2,1


In [46]:
from sklearn.preprocessing import StandardScaler
num_col = ["ApplicantIncome","CoapplicantIncome","LoanAmount" ,"Loan_Amount_Term"]
scaler = StandardScaler()
from sklearn.preprocessing import StandardScaler

In [47]:
X[num_col]

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
0,0.711630,0.092069,0.805980,0.285826
1,-0.398856,-0.539332,-1.350425,0.285826
2,-0.691384,0.447965,0.527735,0.285826
3,1.705666,-0.539332,1.258130,0.285826
4,-0.866761,0.095418,-0.341784,0.285826
...,...,...,...,...
376,1.497318,-0.539332,0.805980,0.285826
377,-0.236106,0.277135,0.110366,0.285826
378,-0.469007,-0.539332,-1.176522,0.285826
379,0.377011,-0.539332,-2.254724,-2.337910


In [48]:
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [62]:
def evaluate_model(model):
    X_train , X_test , Y_train , Y_test = train_test_split(X,Y, test_size = 0.2 ,random_state = 42)
    model.fit(X_train , Y_train)
    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test,Y_pred)
    cross_val = cross_val_score(model,X,Y,cv=5)
    avg_cross_val = np.mean(cross_val)
    print(f"{model.__class__.__name__}- Accuracy : {accuracy : .2f} , Cross-val_Score: {avg_cross_val}")
    return avg_cross_val

In [63]:

models = {
    LogisticRegression(),
    svm.SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),}
    

In [64]:
model_score = {model.__class__.__name__:evaluate_model(model) for model in models}

LogisticRegression- Accuracy :  0.85 , Cross-val_Score: 0.8325899843505477
RandomForestClassifier- Accuracy :  0.83 , Cross-val_Score: 0.812793427230047
GradientBoostingClassifier- Accuracy :  0.85 , Cross-val_Score: 0.8044992175273865
SVC- Accuracy :  0.86 , Cross-val_Score: 0.8353286384976526
DecisionTreeClassifier- Accuracy :  0.86 , Cross-val_Score: 0.7373630672926448


In [72]:
def tune_model(model, param_grid):
    tunner = RandomizedSearchCV(model , param_grid , cv = 5 , n_iter =20, verbose = True , random_state = 42)
    tunner.fit(X,Y)
    print(f"Best score for {model.__class__.__name__}:{tunner.best_score_: .2f}")
    print(f"Best parameter for {model.__class__.__name__}:{tunner.best_params_}")
    return tunner.best_estimator_

In [73]:
log_reg_grid = {'C': np.logspace(-4,4,20), "solver":["liblinear"]}
svc_grid = {'C': [0.25, 0.50 , 0.75 , 1],"kernel":['linear']}

rf_grid = {
    'n_estimators': np.arange(10,1000,10),
    'max_features': ['log2','sqrt'],
    'max_depth': [None , 3,5,10,20,30],
    'min_samples_split': [2,5,20,50,100],
    'min_samples_leaf': [1,2,5,10]
}

In [74]:
best_log_reg = tune_model(LogisticRegression(), log_reg_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best score for LogisticRegression: 0.84
Best parameter for LogisticRegression:{'solver': 'liblinear', 'C': 0.23357214690901212}


In [75]:
best_svc_reg = tune_model(svm.SVC(),svc_grid)

C:\Users\Siddhi\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best score for SVC: 0.84
Best parameter for SVC:{'kernel': 'linear', 'C': 0.25}


In [78]:
best_rf = tune_model(RandomForestClassifier(),rf_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best score for RandomForestClassifier: 0.84
Best parameter for RandomForestClassifier:{'n_estimators': 930, 'min_samples_split': 50, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': 30}


In [79]:
final_model = best_rf

In [80]:
joblib.dump(final_model,'loan_status_predictor.pkl')

['loan_status_predictor.pkl']

In [81]:
#prediction system 
sample_data = pd.DataFrame({'Gender':[1], 'Married': [1],'Dependents':[2],'Education':[0],'Self_Employed':[0],'ApplicantIncome':[2889],	'CoapplicantIncome':[0.0],'LoanAmount':[45],'Loan_Amount_Term':[180],'Credit_History':[1],'Property_Area':[1]})
sample_data[num_col] = scaler.transform(sample_data[num_col])
loaded_model = joblib.load('loan_status_predictor.pkl')
prediction = loaded_model.predict(sample_data)

result = "Loan Approved" if prediction[0]==1 else "Loan Not Approved"
print(f"\nPrediction Result: {result}")
    


Prediction Result: Loan Approved


In [82]:
joblib.dump(scaler,'vector.pkl')

['vector.pkl']